In [21]:
import numpy as np
import pandas as pd
from scipy.stats import norm

###############################################################################
# CONFIG (UPDATED)
###############################################################################
EXCEL_PATH = r"C:\Users\Tom\Git Repos\tomd6464\Risk_1\MCS_Bond_Sim\MCS_Bond.xlsx"

ALPHA     = 0.99
SEED      = 1
PD_FLOOR  = 1e-50
TILT      = 0.8

# "Lots of simulations" but feasible:
MIN_SIMS_BRUTE = 50_000_000        # guaranteed minimum sims (50m)
MAX_SIMS_BRUTE = 2_000_000_000       # hard cap (2b)

MIN_SIMS_IS    = 50_000_000        # guaranteed minimum sims (50m)
MAX_SIMS_IS    = 2_000_000_000     # hard cap (2b)

CHUNK = 5_000_000                  # fewer Python loop iterations

# Early stopping based on relative standard error of Expected Loss estimate
# (lower = more accurate, higher = faster)
TARGET_REL_SE_BRUTE = 0.01         # 1% relative SE target for EL
TARGET_REL_SE_IS    = 0.01         # 1% relative SE target for EL (under true measure)

PRINT_EVERY_CHUNKS  = 10

###############################################################################
# LOAD DATA FROM EXCEL
###############################################################################
def load_bond_data(excel_path):
    raw = pd.read_excel(excel_path, sheet_name="Bond_Data", header=None)

    issuer_row = raw.index[raw[1].astype(str).str.strip() == "Issuer"][0]
    names = raw.loc[issuer_row, 2:4].tolist()

    def get_row(label):
        r = raw.index[raw[1].astype(str).str.strip() == label][0]
        return raw.loc[r, 2:4].astype(float).to_numpy()

    payoff_nd = get_row("Payoff N.d.")
    payoff_d  = get_row("Payoff (D)")

    kmv_row = raw.index[raw[1].astype(str).str.strip() == "KMV Output"][0]
    pd_1y = raw.loc[kmv_row + 1, 2:4].astype(float).to_numpy()

    return names, payoff_nd, payoff_d, pd_1y


def load_corr(excel_path):
    df = pd.read_excel(excel_path, sheet_name="Bond_Stock_data", header=1)
    norm_cols = [c for c in df.columns if str(c).endswith("Return.1")]

    if len(norm_cols) < 3:
        raise ValueError(f"Couldn't find 3 normalised return columns. Found: {norm_cols}")

    corr = df[norm_cols].dropna().corr().to_numpy()
    corr = (corr + corr.T) / 2.0
    np.fill_diagonal(corr, 1.0)
    return corr, norm_cols


###############################################################################
# BRUTE FORCE (STREAMING) — NO HUGE LOSS ARRAY
###############################################################################
def mc_bruteforce_streaming(
    payoff_nd, payoff_d, pd_1y, corr,
    alpha, seed, pd_floor,
    min_sims, max_sims, chunk,
    target_rel_se, print_every_chunks=10
):
    rng = np.random.default_rng(seed)

    payoff_nd = np.asarray(payoff_nd, dtype=np.float32)
    payoff_d  = np.asarray(payoff_d,  dtype=np.float32)
    pd_1y     = np.asarray(pd_1y,     dtype=np.float32)
    corr      = np.asarray(corr,      dtype=np.float32)

    n = len(payoff_nd)
    base = float(payoff_nd.sum())

    L = np.linalg.cholesky(corr).astype(np.float32)
    zcrit = norm.ppf(np.clip(pd_1y, pd_floor, 1 - 1e-12)).astype(np.float32)

    # Streaming accumulators for EL and P(L>0)
    sims_done = 0
    sum_L = 0.0
    sum_L2 = 0.0
    count_pos = 0

    chunks_done = 0
    while sims_done < max_sims:
        m = min(chunk, max_sims - sims_done)

        z  = rng.standard_normal(size=(m, n)).astype(np.float32)
        zc = z @ L.T

        default = (zc < zcrit)
        payoff = np.where(default, payoff_d, payoff_nd)  # vectorized
        port_payoff = payoff.sum(axis=1, dtype=np.float32)
        loss = (base - port_payoff).astype(np.float32)

        # update streaming stats
        sims_done += m
        chunks_done += 1

        loss64 = loss.astype(np.float64)  # for stable accumulation
        sum_L  += float(loss64.sum())
        sum_L2 += float((loss64 * loss64).sum())
        count_pos += int((loss > 0).sum())

        # early stop based on relative SE of mean(loss)
        if sims_done >= min_sims:
            mean = sum_L / sims_done
            # sample variance of loss
            var = max(0.0, (sum_L2 - sims_done * mean * mean) / max(1, sims_done - 1))
            se = (var / sims_done) ** 0.5
            rel_se = se / (abs(mean) + 1e-30)

            if rel_se <= target_rel_se:
                break

        if print_every_chunks and (chunks_done % print_every_chunks == 0):
            mean = sum_L / sims_done
            ppos = count_pos / sims_done
            print(f"[BRUTE] sims={sims_done:,}  EL≈{mean:.6g}  P(L>0)≈{ppos:.6g}")

    EL = sum_L / sims_done
    p_pos = count_pos / sims_done
    p_tail = 1.0 - alpha

    # In your rare-event setup, VaR often 0; we can at least return this correctly:
    if p_pos < p_tail:
        VaR = 0.0
        ES = EL / p_tail if p_tail > 0 else 0.0
        note = "Rare-event regime: VaR=0, ES=EL/(1-alpha)."
    else:
        VaR = None
        ES = None
        note = "P(L>0) >= 1-alpha; need quantile method if you truly need VaR/ES."

    return {
        "method": "brute_force_streaming",
        "VaR": VaR,
        "ES": ES,
        "Expected_Loss": EL,
        "P(Loss>0)": p_pos,
        "alpha": alpha,
        "simulations": sims_done,
        "note": note
    }


###############################################################################
# IMPORTANCE SAMPLING (STREAMING) — ADD EARLY STOP + FASTER MATH
###############################################################################
def mc_importance_sampling_streaming(
    payoff_nd, payoff_d, pd_1y, corr,
    alpha, seed, pd_floor, tilt,
    min_sims, max_sims, chunk,
    target_rel_se, print_every_chunks=10
):
    rng = np.random.default_rng(seed)

    payoff_nd = np.asarray(payoff_nd, dtype=np.float32)
    payoff_d  = np.asarray(payoff_d,  dtype=np.float32)
    pd_1y     = np.asarray(pd_1y,     dtype=np.float32)
    corr      = np.asarray(corr,      dtype=np.float32)

    n = len(payoff_nd)
    base = float(payoff_nd.sum())

    L = np.linalg.cholesky(corr).astype(np.float32)
    zcrit = norm.ppf(np.clip(pd_1y, pd_floor, 1 - 1e-12)).astype(np.float32)

    # shift (tilt)
    mu = (tilt * zcrit).astype(np.float32)
    mu2 = float(np.dot(mu, mu))  # scalar

    # Streaming accumulators under true measure:
    sims_done = 0
    w_sum = 0.0
    wL_sum = 0.0
    wL2_sum = 0.0      # for SE of weighted estimator
    w_pos_sum = 0.0
    defaults_seen_tilted = 0

    chunks_done = 0
    while sims_done < max_sims:
        m = min(chunk, max_sims - sims_done)

        z  = rng.standard_normal(size=(m, n)).astype(np.float32)
        zc = z @ L.T

        zc_tilt = zc + mu
        default = (zc_tilt < zcrit)
        defaults_seen_tilted += int(default.any(axis=1).sum())

        payoff = np.where(default, payoff_d, payoff_nd)
        port_payoff = payoff.sum(axis=1, dtype=np.float32)
        loss = (base - port_payoff).astype(np.float32)

        # weights (compute in float64 for stability)
        # dot each row with mu:
        dot = (zc.astype(np.float64) @ mu.astype(np.float64))
        logw = -(dot) - 0.5 * mu2
        w = np.exp(logw)  # float64

        loss64 = loss.astype(np.float64)

        # update sums
        sims_done += m
        chunks_done += 1

        w_sum   += float(w.sum())
        wL      = w * loss64
        wL_sum  += float(wL.sum())
        wL2_sum += float((wL * loss64).sum())  # E[w*L^2] numerator-ish for SE approx
        w_pos_sum += float(w[loss > 0].sum())

        # early stop based on approximate relative SE of EL = (sum wL)/(sum w)
        if sims_done >= min_sims and w_sum > 0:
            EL = wL_sum / w_sum

            # crude but useful SE approximation:
            # Var_hat ≈ (E[w*L^2]/E[w]) - EL^2 ; SE ≈ sqrt(Var_hat / ESS)
            # ESS ≈ (sum w)^2 / sum(w^2)
            w2_sum = float((w * w).sum())
            ESS = (w_sum * w_sum) / (w2_sum + 1e-300)

            EwL2_over_Ew = (wL2_sum / w_sum) if w_sum > 0 else 0.0
            var_hat = max(0.0, EwL2_over_Ew - EL * EL)
            se = (var_hat / max(1.0, ESS)) ** 0.5
            rel_se = se / (abs(EL) + 1e-30)

            if rel_se <= target_rel_se:
                break

        if print_every_chunks and (chunks_done % print_every_chunks == 0) and w_sum > 0:
            EL = wL_sum / w_sum
            ppos = w_pos_sum / w_sum
            print(f"[IS] sims={sims_done:,}  EL≈{EL:.6g}  P(L>0)≈{ppos:.6g}")

    EL = wL_sum / w_sum if w_sum > 0 else 0.0
    p_pos = w_pos_sum / w_sum if w_sum > 0 else 0.0
    p_tail = 1.0 - alpha

    if p_pos < p_tail:
        VaR = 0.0
        ES = EL / p_tail if p_tail > 0 else 0.0
        note = "Rare-event regime: VaR=0, ES=EL/(1-alpha)."
    else:
        VaR = None
        ES = None
        note = "P(L>0) >= 1-alpha; need weighted-quantile method if you truly need VaR/ES."

    return {
        "method": "importance_sampling_streaming",
        "VaR": VaR,
        "ES": ES,
        "Expected_Loss": EL,
        "P(Loss>0)": p_pos,
        "alpha": alpha,
        "simulations": sims_done,
        "tilt_strength": tilt,
        "tilted_default_paths_seen": defaults_seen_tilted,
        "note": note
    }


###############################################################################
# MAIN
###############################################################################
if __name__ == "__main__":
    names, payoff_nd, payoff_d, pd_1y = load_bond_data(EXCEL_PATH)
    corr, cols = load_corr(EXCEL_PATH)

    print("="*70)
    print("LOADED DATA FROM EXCEL")
    print("="*70)
    print(f"\nIssuers: {names}")
    print(f"\nPayoff (No Default): {payoff_nd}")
    print(f"Payoff (Default):    {payoff_d}")
    
    print(f"\n[CRITICAL] PDs loaded from Excel:")
    for i, (name, pd) in enumerate(zip(names, pd_1y)):
        print(f"  {name}: {pd:.10e}")
    print(f"Sum of PDs: {float(np.sum(pd_1y)):.10e}")
    
    print(f"\nCorrelation columns found: {cols}")
    print(f"Correlation matrix shape: {corr.shape}")
    print(f"Correlation matrix:\n{corr}")
    
    brute = mc_bruteforce_streaming(
        payoff_nd, payoff_d, pd_1y, corr,
        alpha=ALPHA, seed=SEED, pd_floor=PD_FLOOR,
        min_sims=MIN_SIMS_BRUTE, max_sims=MAX_SIMS_BRUTE, chunk=CHUNK,
        target_rel_se=TARGET_REL_SE_BRUTE, print_every_chunks=PRINT_EVERY_CHUNKS
    )

    print("\n--- BRUTE FORCE (STREAMING) ---")
    for k, v in brute.items():
        print(f"{k}: {v}")

    is_stats = mc_importance_sampling_streaming(
        payoff_nd, payoff_d, pd_1y, corr,
        alpha=ALPHA, seed=SEED + 1, pd_floor=PD_FLOOR, tilt=TILT,
        min_sims=MIN_SIMS_IS, max_sims=MAX_SIMS_IS, chunk=CHUNK,
        target_rel_se=TARGET_REL_SE_IS, print_every_chunks=PRINT_EVERY_CHUNKS
    )

    print("\n--- IMPORTANCE SAMPLING (STREAMING) ---")
    for k, v in is_stats.items():
        print(f"{k}: {v}")



LOADED DATA FROM EXCEL

Issuers: ['NVR US Equity', 'EBAY US Equity', 'MSI US Equity']

Payoff (No Default): [1096.89213894  993.07854348  885.79231146]
Payoff (Default):    [-15914.54602848 -15914.76145403 -14668.07900684]

[CRITICAL] PDs loaded from Excel:
  NVR US Equity: 2.1410517647e-38
  EBAY US Equity: 2.0575996750e-15
  MSI US Equity: 2.0367921874e-37
Sum of PDs: 2.0575996750e-15

Correlation columns found: ['NVR Return.1', 'EBAY Return.1', 'MSI Return.1']
Correlation matrix shape: (3, 3)
Correlation matrix:
[[1.         0.33756865 0.4482751 ]
 [0.33756865 1.         0.35468667]
 [0.4482751  0.35468667 1.        ]]

--- BRUTE FORCE (STREAMING) ---
method: brute_force_streaming
VaR: 0.0
ES: 0.0
Expected_Loss: 0.0
P(Loss>0): 0.0
alpha: 0.99
simulations: 50000000
note: Rare-event regime: VaR=0, ES=EL/(1-alpha).
[IS] sims=50,000,000  EL≈3.49033e-27  P(L>0)≈2.06432e-31
[IS] sims=100,000,000  EL≈3.63149e-27  P(L>0)≈2.14781e-31
[IS] sims=150,000,000  EL≈3.85854e-27  P(L>0)≈2.2821e-31
[

In [20]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')  # Non-interactive backend
import matplotlib.pyplot as plt
from scipy.stats import norm

###############################################################################
# CONFIG
###############################################################################
EXCEL_PATH = r"C:\Users\Tom\Git Repos\tomd6464\Risk_1\MCS_Bond_Sim\MCS_Bond.xlsx"
SEED = 1
PD_FLOOR = 1e-50

###############################################################################
# LOAD DATA FROM EXCEL
###############################################################################
def load_bond_data(excel_path):
    raw = pd.read_excel(excel_path, sheet_name="Bond_Data", header=None)
    issuer_row = raw.index[raw[1].astype(str).str.strip() == "Issuer"][0]
    names = raw.loc[issuer_row, 2:4].tolist()

    def get_row(label):
        r = raw.index[raw[1].astype(str).str.strip() == label][0]
        return raw.loc[r, 2:4].astype(float).to_numpy()

    payoff_nd = get_row("Payoff N.d.")
    payoff_d  = get_row("Payoff (D)")
    kmv_row = raw.index[raw[1].astype(str).str.strip() == "KMV Output"][0]
    pd_1y = raw.loc[kmv_row + 1, 2:4].astype(float).to_numpy()

    return names, payoff_nd, payoff_d, pd_1y


def load_corr(excel_path):
    df = pd.read_excel(excel_path, sheet_name="Bond_Stock_data", header=1)
    norm_cols = [c for c in df.columns if str(c).endswith("Return.1")]

    if len(norm_cols) < 3:
        raise ValueError(f"Couldn't find 3 normalised return columns. Found: {norm_cols}")

    corr = df[norm_cols].dropna().corr().to_numpy()
    corr = (corr + corr.T) / 2.0
    np.fill_diagonal(corr, 1.0)
    return corr, norm_cols


###############################################################################
# VISUALIZATION FUNCTIONS
###############################################################################

def plot_convergence_streaming(
    payoff_nd, payoff_d, pd_1y, corr,
    seed, pd_floor,
    max_sims=10_000_000,
    chunk=500_000,
    out_png="Fig_Convergence.png"
):
    """Track EL and P(Loss>0) convergence across simulation batches (downsampled for plotting)."""
    rng = np.random.default_rng(seed)

    payoff_nd = np.asarray(payoff_nd, dtype=np.float32)
    payoff_d  = np.asarray(payoff_d,  dtype=np.float32)
    pd_1y     = np.asarray(pd_1y,     dtype=np.float32)
    corr      = np.asarray(corr,      dtype=np.float32)

    n = len(payoff_nd)
    base = float(payoff_nd.sum())

    L = np.linalg.cholesky(corr).astype(np.float32)
    zcrit = norm.ppf(np.clip(pd_1y, pd_floor, 1 - 1e-12)).astype(np.float32)

    sims_log = []
    el_log = []
    ppos_log = []

    sims_done = 0
    sum_L = 0.0
    count_pos = 0
    last_logged = 0

    while sims_done < max_sims:
        m = min(chunk, max_sims - sims_done)

        z  = rng.standard_normal(size=(m, n)).astype(np.float32)
        zc = z @ L.T

        default = (zc < zcrit)
        payoff = np.where(default, payoff_d, payoff_nd)
        port_payoff = payoff.sum(axis=1, dtype=np.float32)
        loss = (base - port_payoff).astype(np.float32)

        sims_done += m
        sum_L  += float(loss.sum())
        count_pos += int((loss > 0).sum())

        if sims_done - last_logged >= chunk * 5 or sims_done >= max_sims:
            sims_log.append(sims_done)
            el_log.append(sum_L / sims_done)
            ppos_log.append(count_pos / sims_done)
            last_logged = sims_done

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), dpi=100)

    ax1.plot(np.array(sims_log) / 1e6, el_log, linewidth=2, marker='o', markersize=4)
    ax1.set_xlabel("Simulations (millions)")
    ax1.set_ylabel("Expected Loss")
    ax1.set_title("Convergence of Expected Loss")
    ax1.grid(True, alpha=0.3)

    ax2.plot(np.array(sims_log) / 1e6, ppos_log, linewidth=2, marker='o', markersize=4, color='orange')
    ax2.set_xlabel("Simulations (millions)")
    ax2.set_ylabel("P(Loss > 0)")
    ax2.set_title("Convergence of Default Probability")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(out_png, dpi=100, bbox_inches='tight')
    plt.close(fig)

    return {"out_png": out_png, "final_el": el_log[-1], "final_ppos": ppos_log[-1], "points_plotted": len(sims_log)}


def plot_simulation_comparison(brute_stats, is_stats, out_png="Fig_SimulationComparison.png"):
    """Bar chart comparing brute force vs importance sampling effort."""
    methods = ["Brute Force", "Importance Sampling"]
    sims = [brute_stats["simulations"], is_stats["simulations"]]
    colors = ["#1f77b4", "#ff7f0e"]

    fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
    bars = ax.bar(methods, np.array(sims) / 1e6, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

    for bar, s in zip(bars, sims):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height, f'{s/1e6:.1f}M', 
                ha='center', va='bottom', fontsize=11, fontweight='bold')

    ax.set_ylabel("Number of Simulations (millions)", fontsize=12)
    ax.set_title("Simulation Effort: Brute Force vs Importance Sampling", fontsize=13)
    ax.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig(out_png, dpi=100, bbox_inches='tight')
    plt.close(fig)

    return {"out_png": out_png}


def plot_default_analysis(pd_1y, corr, out_png="Fig_DefaultAnalysis.png"):
    """Visualize why defaults are rare."""
    n_issuers = len(pd_1y)
    
    fig = plt.figure(figsize=(11, 8), dpi=100)
    gs = fig.add_gridspec(2, 2, hspace=0.35, wspace=0.35)

    # Plot 1: Individual PDs
    ax1 = fig.add_subplot(gs[0, 0])
    issuer_labels = [f"Issuer {i+1}" for i in range(n_issuers)]
    ax1.bar(issuer_labels, pd_1y * 100, color='#d62728', alpha=0.7, edgecolor='black')
    ax1.set_ylabel("PD (%)", fontsize=10)
    ax1.set_title("Default Probabilities", fontsize=11, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')

    # Plot 2: Correlation matrix heatmap
    ax2 = fig.add_subplot(gs[0, 1])
    if n_issuers > 1:
        im = ax2.imshow(corr, cmap='coolwarm', vmin=0, vmax=1, aspect='auto')
        ax2.set_xticks(range(len(issuer_labels)))
        ax2.set_yticks(range(len(issuer_labels)))
        ax2.set_xticklabels(issuer_labels, fontsize=9)
        ax2.set_yticklabels(issuer_labels, fontsize=9)
        ax2.set_title("Correlation Matrix", fontsize=11, fontweight='bold')
        plt.colorbar(im, ax=ax2, label='ρ')
    else:
        ax2.text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=14)
        ax2.set_title("Correlation (N/A)", fontsize=11, fontweight='bold')
        ax2.axis('off')

    # Plot 3: Joint default probability
    ax3 = fig.add_subplot(gs[1, 0])
    joint_ind = np.prod(pd_1y)
    
    if n_issuers > 1:
        mean_corr = np.mean(np.triu(corr, 1))
        scenarios = ['Independent', f'Correlated']
        joint_probs = [joint_ind * 100, joint_ind * (1 + mean_corr) * 100]
    else:
        scenarios = ['Default Prob']
        joint_probs = [joint_ind * 100]

    ax3.bar(scenarios, joint_probs, color=['#2ca02c', '#ff7f0e'][:len(scenarios)], alpha=0.7, edgecolor='black')
    ax3.set_ylabel("Probability (%)", fontsize=10)
    ax3.set_title("Joint Default Analysis", fontsize=11, fontweight='bold')
    ax3.set_yscale('log')

    # Plot 4: Summary text
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.axis('off')
    
    summary_text = (
        f"Summary\n"
        f"Max PD: {np.max(pd_1y)*100:.4f}%\n"
        f"Min PD: {np.min(pd_1y)*100:.4f}%\n\n"
        f"Very low PDs mean\n"
        f"defaults are rare.\n"
        f"MC needs many sims."
    )
    ax4.text(0.05, 0.95, summary_text, transform=ax4.transAxes,
            fontsize=9, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.savefig(out_png, dpi=100, bbox_inches='tight')
    plt.close(fig)

    return {"out_png": out_png}


###############################################################################
# RUN ALL VISUALIZATIONS
###############################################################################
names, payoff_nd, payoff_d, pd_1y = load_bond_data(EXCEL_PATH)
corr, cols = load_corr(EXCEL_PATH)

print("\n" + "="*70)
print("GENERATING DIAGNOSTIC VISUALIZATIONS")
print("="*70)

try:
    max_sims_to_plot = brute['simulations']
except NameError:
    print("\n⚠ WARNING: Run cell 1 first to generate simulation results.")
    max_sims_to_plot = 10_000_000

conv = plot_convergence_streaming(payoff_nd, payoff_d, pd_1y, corr, SEED, PD_FLOOR, max_sims_to_plot, 500_000, "Fig_Convergence.png")
print(f"✓ Convergence: {conv['out_png']}")

try:
    comp = plot_simulation_comparison(brute, is_stats, "Fig_SimulationComparison.png")
    print(f"✓ Comparison: {comp['out_png']}")
except NameError:
    print("⚠ Skipping comparison. Run cell 1 first.")

deflt = plot_default_analysis(pd_1y, corr, "Fig_DefaultAnalysis.png")
print(f"✓ Analysis: {deflt['out_png']}")

print("="*70)


GENERATING DIAGNOSTIC VISUALIZATIONS
✓ Convergence: Fig_Convergence.png
✓ Comparison: Fig_SimulationComparison.png
✓ Analysis: Fig_DefaultAnalysis.png


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy.stats import norm

###############################################################################
# CONFIG
###############################################################################
EXCEL_PATH = r"C:\Users\Tom\Git Repos\tomd6464\Risk_1\MCS_Bond_Sim\MCS_Bond.xlsx"
SEED = 1
PD_FLOOR = 1e-50

###############################################################################
# LOAD DATA FROM EXCEL
###############################################################################
def load_bond_data(excel_path):
    raw = pd.read_excel(excel_path, sheet_name="Bond_Data", header=None)
    issuer_row = raw.index[raw[1].astype(str).str.strip() == "Issuer"][0]
    names = raw.loc[issuer_row, 2:4].tolist()

    def get_row(label):
        r = raw.index[raw[1].astype(str).str.strip() == label][0]
        return raw.loc[r, 2:4].astype(float).to_numpy()

    payoff_nd = get_row("Payoff N.d.")
    payoff_d  = get_row("Payoff (D)")
    kmv_row = raw.index[raw[1].astype(str).str.strip() == "KMV Output"][0]
    pd_1y = raw.loc[kmv_row + 1, 2:4].astype(float).to_numpy()

    return names, payoff_nd, payoff_d, pd_1y


def load_corr(excel_path):
    df = pd.read_excel(excel_path, sheet_name="Bond_Stock_data", header=1)
    norm_cols = [c for c in df.columns if str(c).endswith("Return.1")]

    if len(norm_cols) < 3:
        raise ValueError(f"Couldn't find 3 normalised return columns. Found: {norm_cols}")

    corr = df[norm_cols].dropna().corr().to_numpy()
    corr = (corr + corr.T) / 2.0
    np.fill_diagonal(corr, 1.0)
    return corr, norm_cols


###############################################################################
# PROBABILITY OF LOSS CONVERGENCE TRACKING
###############################################################################
def plot_plos_convergence_detailed(
    payoff_nd, payoff_d, pd_1y, corr,
    seed, pd_floor,
    max_sims=100_000_000,
    chunk=500_000,
    out_png="Fig_PLoss_Convergence.png"
):
    """Track P(Loss>0) with detailed milestone markers and confidence bands."""
    rng = np.random.default_rng(seed)

    payoff_nd = np.asarray(payoff_nd, dtype=np.float32)
    payoff_d  = np.asarray(payoff_d,  dtype=np.float32)
    pd_1y     = np.asarray(pd_1y,     dtype=np.float32)
    corr      = np.asarray(corr,      dtype=np.float32)

    n = len(payoff_nd)
    base = float(payoff_nd.sum())

    L = np.linalg.cholesky(corr).astype(np.float32)
    zcrit = norm.ppf(np.clip(pd_1y, pd_floor, 1 - 1e-12)).astype(np.float32)

    sims_log = []
    plos_log = []
    plos_se_log = []

    sims_done = 0
    count_pos = 0

    while sims_done < max_sims:
        m = min(chunk, max_sims - sims_done)

        z  = rng.standard_normal(size=(m, n)).astype(np.float32)
        zc = z @ L.T

        default = (zc < zcrit)
        payoff = np.where(default, payoff_d, payoff_nd)
        port_payoff = payoff.sum(axis=1, dtype=np.float32)
        loss = (base - port_payoff).astype(np.float32)

        sims_done += m
        count_pos += int((loss > 0).sum())

        p_los = count_pos / sims_done
        se_plos = np.sqrt(p_los * (1 - p_los) / sims_done)
        
        sims_log.append(sims_done)
        plos_log.append(p_los)
        plos_se_log.append(se_plos)

    sims_array = np.array(sims_log) / 1e6
    plos_array = np.array(plos_log)
    se_array = np.array(plos_se_log)

    upper_band = plos_array + 1.96 * se_array
    lower_band = np.maximum(plos_array - 1.96 * se_array, 0)
    final_plos = plos_array[-1]

    # Figure 1: Linear scale with zoomed y-axis
    fig1, ax1 = plt.subplots(figsize=(12, 7), dpi=100)
    ax1.plot(sims_array, plos_array, linewidth=2.5, color='#1f77b4', label='P(Loss > 0)', zorder=3)
    ax1.fill_between(sims_array, lower_band, upper_band, alpha=0.2, color='#1f77b4', label='95% Confidence Band', zorder=1)
    
    ax1.axhline(y=final_plos, color='red', linestyle='--', linewidth=1.5, alpha=0.7, 
                label=f'Final: {final_plos:.6e}')
    
    # Set y-axis limits to the observed range with 10% padding
    y_min = min(lower_band) * 0.9
    y_max = max(upper_band) * 1.1
    ax1.set_ylim([y_min, y_max])
    
    ax1.set_xlabel("Simulations (millions)", fontsize=12, fontweight='bold')
    ax1.set_ylabel("P(Loss > 0)", fontsize=12, fontweight='bold')
    ax1.set_title("P(Loss > 0) Convergence (Linear Scale - Zoomed to Data)", fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(fontsize=11, loc='best')
    ax1.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    
    plt.tight_layout()
    plt.savefig(out_png, dpi=100, bbox_inches='tight')
    plt.close(fig1)

    # Figure 2: Log-log scale
    fig2, ax2 = plt.subplots(figsize=(12, 7), dpi=100)
    
    mask = plos_array > 0
    ax2.loglog(sims_array[mask], plos_array[mask], linewidth=2.5, color='#1f77b4', 
               marker='o', markersize=3, label='P(Loss > 0)', zorder=3)
    
    ax2.fill_between(sims_array[mask], 
                     np.maximum(lower_band[mask], 1e-35),
                     upper_band[mask], 
                     alpha=0.2, color='#1f77b4', label='95% Confidence Band', zorder=1)
    
    ax2.axhline(y=final_plos, color='red', linestyle='--', linewidth=1.5, alpha=0.7, 
                label=f'Final: {final_plos:.6e}')
    
    ax2.set_xlabel("Simulations (millions, log scale)", fontsize=12, fontweight='bold')
    ax2.set_ylabel("P(Loss > 0) (log scale)", fontsize=12, fontweight='bold')
    ax2.set_title("P(Loss > 0) Convergence (Log-Log Scale)", fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--', which='both')
    ax2.legend(fontsize=11, loc='best')
    
    plt.tight_layout()
    plt.savefig(out_png.replace('.png', '_loglog.png'), dpi=100, bbox_inches='tight')
    plt.close(fig2)

    return {
        "out_png_linear": out_png,
        "out_png_loglog": out_png.replace('.png', '_loglog.png'),
        "final_plos": final_plos,
        "final_se": se_array[-1],
        "total_sims": sims_done,
        "y_min": y_min,
        "y_max": y_max
    }


###############################################################################
# RUN CONVERGENCE ANALYSIS
###############################################################################
names, payoff_nd, payoff_d, pd_1y = load_bond_data(EXCEL_PATH)
corr, cols = load_corr(EXCEL_PATH)

print("\n" + "="*70)
print("P(LOSS > 0) CONVERGENCE ANALYSIS")
print("="*70)

conv_plos = plot_plos_convergence_detailed(
    payoff_nd, payoff_d, pd_1y, corr,
    seed=SEED, pd_floor=PD_FLOOR,
    max_sims=100_000_000,
    chunk=500_000,
    out_png="Fig_PLoss_Convergence.png"
)

print(f"\n✓ Linear Scale Chart: {conv_plos['out_png_linear']}")
print(f"✓ Log-Log Scale Chart: {conv_plos['out_png_loglog']}")
print(f"  Total Simulations: {conv_plos['total_sims']:,}")
print(f"  Final P(Loss > 0): {conv_plos['final_plos']:.6e}")
print(f"  Y-axis range: [{conv_plos['y_min']:.6e}, {conv_plos['y_max']:.6e}]")
print(f"  Final Std Error: {conv_plos['final_se']:.6e}")

print("\n" + "="*70)


P(LOSS > 0) CONVERGENCE ANALYSIS


C:\Users\Tom\AppData\Local\Temp\ipykernel_19680\905868304.py:151: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.tight_layout()



✓ Linear Scale Chart: Fig_PLoss_Convergence.png
✓ Log-Log Scale Chart: Fig_PLoss_Convergence_loglog.png
  Total Simulations: 100,000,000
  Final P(Loss > 0): 0.00000000%
  Final Std Error: 0.00000000%

